# Reporting Dashboard Setup for Tableau

This notebook prepares the final engineered dataset for Tableau visualization by loading it into PostgreSQL. Tableau will then connect to this data source to build interactive fraud detection dashboards, enabling stakeholders to explore fraud trends, risk scores, and claim characteristics with ease.

> **PLEASE READ BEFORE PROCEEDING:**  
> The steps from 1-3 are for users with **Tableau Desktop** only and not recommended for users with **Tableau Public**.  
> For **Tableau Public** users, please refer to steps 4.  

In [1]:
# Imports essential libraries and Setup
import pandas as pd
import os
from sqlalchemy import create_engine, text
from getpass import getpass  # For secure password input

# Project directory and engineered features file path
project_dir = r"C:\Users\Cloud\OneDrive\Desktop\Fraud_Analytics_Project"
feature_file = os.path.join(project_dir, "data", "features", "engineered_insurance_claims.csv")
print("Imports and file path setup complete.")

Imports and file path setup complete.


### 1. PostgreSQL Connection Setup

I created a secure connection to the PostgreSQL database where Tableau will fetch the data.  
Using `getpass()` avoids hardcoding sensitive passwords in code.

In [2]:
# Securely prompt for PostgreSQL password
pg_user = "postgres"
pg_password = getpass("Enter PostgreSQL password: ")
pg_host = "localhost"
pg_port = 5432
pg_dbname = "fraud_analytics"

# Create SQLAlchemy engine for PostgreSQL connection
engine = create_engine(f"postgresql+psycopg2://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_dbname}")
print("PostgreSQL engine created.")

PostgreSQL engine created.


### 2. Load Engineered Data into PostgreSQL

Load the final feature-engineered dataset into a PostgreSQL table called `tableau_insurance_claims`.  
Using `if_exists='replace'` ensures the table refreshes with the latest data on each run, keeping Tableau dashboards up-to-date.

In [3]:
# Drop the view if it exists using SQLAlchemy's `text()`
with engine.connect() as conn:
    conn.execute(text("DROP VIEW IF EXISTS v_fraud_summary;"))  # Wrapped in text()
    conn.commit()  # Commit the transaction

# Load the engineered dataset into a DataFrame
df = pd.read_csv(feature_file)

# Replace the PostgreSQL table
df.to_sql("tableau_insurance_claims", engine, if_exists="replace", index=False)
print(f"Loaded {df.shape[0]} records into PostgreSQL table: tableau_insurance_claims")

Loaded 1000 records into PostgreSQL table: tableau_insurance_claims


### 3. For Tableau Desktop Users: Manually Create PostgreSQL View

To simplify your data connection and improve dashboard performance in Tableau Desktop, create a PostgreSQL view that organizes key fraud analytics columns.

### Follow these steps to create the view using pgAdmin4:

1. **Open pgAdmin4** on your computer and connect to your PostgreSQL server.

2. In the left sidebar, navigate to your database:
   - Expand **Servers** → your server name → **Databases** → `fraud_analytics` (or your database name).

3. Right-click on **Schemas** → **public** → **Views**, and select **Create** → **View...**

4. In the **Create - View** dialog:
   - Enter the view name as: `v_fraud_summary`
   - Click the **Definition** tab or section.

5. Paste the following SQL query into the **View Definition** box:

    ```sql
    CREATE OR REPLACE VIEW v_fraud_summary AS
    SELECT
        fraud_reported,
        total_claim_amount,
        risk_score,
        fraud_weight,
        incident_year,
        incident_month,
        incident_dayofweek,
        incident_is_weekend,
        "incident_type_PARKED CAR",
        "incident_type_SINGLE VEHICLE COLLISION",
        "incident_type_VEHICLE THEFT",
        "collision_type_REAR COLLISION",
        "collision_type_SIDE COLLISION",
        "police_report_available_YES"
    FROM
    tableau_insurance_claims;
    ```

6. Click **Save** or **OK** to create the view.

7. To verify, expand the **Views** node under your database schema, and confirm `v_fraud_summary` appears.

8. Now, in Tableau, you can connect directly to this view `v_fraud_summary` instead of the full table for easier querying and cleaner dashboards.

---

> Notes:  
> Creating this view does **not** duplicate the data (as tested), it simply defines a saved SQL query that Tableau can use.  
> If data updates frequently, the view will always reflect the latest table data.  

---

### 4. Export Data for Tableau Public

Now for users who only have Tableau Public and since the public version does **not** support direct database connections like PostgreSQL, follow these steps to export the data as a CSV file that you can load into Tableau Public:

1. **Export data from PostgreSQL to CSV:**
   - This notebook already saves the cleaned and engineered data as a CSV (`engineered_insurance_claims.csv`).
   - If you want to export the latest data from the database, you can also run a SQL query in pgAdmin or use a Python script to export the table or view as CSV.

2. **Load CSV into Tableau Public:**
   - Open Tableau Public.
   - Click **“Text File”** under **Connect** on the left sidebar.
   - Browse and select the CSV file (`engineered_insurance_claims.csv`) exported by this project.
   - Once loaded, you can start building your dashboards and visualizations.

3. **Keep your CSV updated:**
   - For good practice, whenever new data or model outputs are available, rerun the export process to refresh your CSV file for Tableau Public.

---

>**Tip:**  
>If you have Tableau Desktop or upgrade later, you can connect directly to the PostgreSQL database (please refer back from steps 1-3 for live data refreshes and more advanced workflows).

---

**Summary:**  
- Export data as CSV from PostgreSQL or this notebook.  
- Load CSV into Tableau Public.  
- Create and publish dashboards from the static CSV data.

---

> **NOTE**  
>The code below is when:  
> - You make further updates or merges after the original CSV.  
> - You want to create a separate CSV specifically for dashboarding (maybe with selected columns or formatting).  
> - You want to automate refreshing the dashboard data by re-exporting after new model runs.  

`Therefore, It’s totally safe to run the code every time you rerun the notebook!  
It simply overwrites (or creates) the CSV file with the latest data — no risk of data corruption or issues.  
In fact, running it as part of your full pipeline ensures your Tableau Public CSV is always up-to-date after any data or model changes.`

In [4]:
# Define export path for Tableau Public
export_csv_path = os.path.join(project_dir, "data", "tableau_insurance_claims_final.csv")

# Export DataFrame to CSV
df.to_csv(export_csv_path, index=False)

print(f"Dataset exported to CSV for Tableau Public: {export_csv_path}")

Dataset exported to CSV for Tableau Public: C:\Users\Cloud\OneDrive\Desktop\Fraud_Analytics_Project\data\tableau_insurance_claims_final.csv


### 5. Dashboard Ideas for Fraud Analytics

- **Total Claim Amount** by Fraud Status  
- **Average Risk Score** by Incident Type  
- Fraud **Trends Over Time** (Monthly, Weekly, Daily)  
- Heatmap of **Fraud Frequency** by Hour of Day vs Collision Type  
- Bar chart showing **Fraud Percentage** by Police Report Availability